# Silver Transformations Samuel L. Jackson movies and series

In [1]:
from pyspark.sql import SparkSession
from IPython.display import display as dis

In [2]:
spark = SparkSession.builder.getOrCreate()

### Load data

In [3]:
movies_cast_df = spark.read.json("../Bronze/samuel_movies_cast.json")
movies_crew_df = spark.read.json("../Bronze/samuel_movies_crew.json")

series_cast_df = spark.read.json("../Bronze/samuel_series_cast.json")
series_crew_df = spark.read.json("../Bronze/samuel_series_crew.json")

genres_df = spark.read.json("../Bronze/tmdb_genres.json")

### Transforming the data

In [4]:
# Removing duplicated genres
genres_df = genres_df.drop_duplicates()

genres_df.show(100)

+-----+------------------+
|   id|              name|
+-----+------------------+
|10766|              Soap|
|10759|Action & Adventure|
|   27|            Horror|
|  878|   Science Fiction|
|   53|          Thriller|
|   36|           History|
|10752|               War|
|10764|           Reality|
|10770|          TV Movie|
|10763|              News|
|10749|           Romance|
|   18|             Drama|
|10762|              Kids|
|10765|  Sci-Fi & Fantasy|
|10751|            Family|
|   80|             Crime|
|10767|              Talk|
|   37|           Western|
|10402|             Music|
|   99|       Documentary|
|10768|    War & Politics|
|   16|         Animation|
|   14|           Fantasy|
| 9648|           Mystery|
|   12|         Adventure|
|   28|            Action|
|   35|            Comedy|
+-----+------------------+



In [5]:
# Removing duplicated series and movies

movies_cast_df = movies_cast_df.drop_duplicates()
movies_crew_df = movies_crew_df.drop_duplicates()

series_cast_df = series_cast_df.drop_duplicates()
series_crew_df = series_crew_df.drop_duplicates()

In [6]:
# The movies and series from 'cast' doesnt have the attributes 'job' and 'department', so Im adding it
from pyspark.sql.functions import lit

movies_cast_df = movies_cast_df.withColumn("job", lit("Actor")).withColumn("department", lit("Cast"))

series_cast_df = series_cast_df.withColumn("job", lit("Actor")).withColumn("department", lit("Cast"))

In [7]:
# The movies doesnt have the attribute order nor character, so Im adding it
movies_crew_df = movies_crew_df.withColumn("order", lit(0)).withColumn("character", lit(None))

# Series crew doest have the 'character' information, adding it...
series_crew_df = series_crew_df.withColumn("character", lit(None))

In [8]:
# Merging the movies cast and crew together in a single dataframe

movies_df = movies_cast_df.unionByName(movies_crew_df)

In [9]:
# Merging the series cast and crew together in a single dataframe

series_df = series_cast_df.unionByName(series_crew_df)

### Saving the dataframes in parquet files

In [10]:
# Adding column 'year' in movies and series to partition it
from pyspark.sql.functions import year, to_date
from pyspark.sql.types import IntegerType

movies_df = movies_df.withColumn("year", year(to_date(movies_df["release_date"], "yyyy-MM-dd")).cast(IntegerType()))

series_df = series_df.withColumn("year", year(to_date(series_df["first_air_date"], "yyyy-MM-dd")).cast(IntegerType()))

In [11]:
movies_df.write.parquet("../Silver/movies/samuel_movies.parquet", mode="overwrite", partitionBy="year")

series_df.write.parquet("../Silver/series/samuel_series.parquet", mode="overwrite", partitionBy="year")

genres_df.write.parquet("../Silver/genres/genres.parquet", mode="overwrite")